---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn import tree

from IPython.display import display

In [3]:
def read_data(data_path):
    df = pd.read_csv(data_path, header=0, encoding='ISO-8859-1', parse_dates=['hearing_date', 'ticket_issued_date'])
    df['interval'] = abs((df['hearing_date'].dt.date - df['ticket_issued_date'].dt.date).astype('timedelta64[D]')).fillna(0)
    return df
    
def clean_data():
    # Opening ddata set
    df_valid = read_data('test.csv')
    df_train = read_data('train.csv')

    # Selecting variables                                                
    df_train = df_train[['ticket_id', 'interval', 'disposition', 'inspector_name', 
                         'judgment_amount', 'fine_amount', 'violation_code', 'compliance']]
    df_train = df_train.dropna(subset=['compliance'])
    df_valid = df_valid[['ticket_id', 'interval', 'disposition', 'inspector_name', 
                         'judgment_amount', 'fine_amount', 'violation_code']]

    # Encoding categorical features
    le = preprocessing.LabelEncoder()
    
    keys = set(list(df_train['disposition'].unique())+list(df_valid['disposition'].unique()))
    le.fit(list(keys))
    df_train['disposition'] = le.transform(df_train['disposition'].values)  
    df_valid['disposition'] = le.transform(df_valid['disposition'].values) 
    
    keys = set(list(df_train['inspector_name'].unique())+list(df_valid['inspector_name'].unique()))
    le.fit(list(keys))
    df_train['inspector_name'] = le.transform(df_train['inspector_name'].values)  
    df_valid['inspector_name'] = le.transform(df_valid['inspector_name'].values) 
    
    keys = set(list(df_train['violation_code'].unique())+list(df_valid['violation_code'].unique()))
    le.fit(list(keys))
    df_train['violation_code'] = le.transform(df_train['violation_code'].values) 
    df_valid['violation_code'] = le.transform(df_valid['violation_code'].values) 
         
    df_train = pd.get_dummies(df_train, columns=['disposition', 'violation_code'])
    df_valid = pd.get_dummies(df_valid, columns=['disposition', 'violation_code'])
        
    df_train, df_valid = df_train.align(df_valid, axis=1)
    df_train.fillna(0, inplace=True)
    df_valid.fillna(0, inplace=True)
                 
    return df_train, df_valid

In [4]:
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

def blight_model():
    df_train, df_valid = clean_data()
    # print(df_train.shape, df_valid.shape)

    X_train = df_train.iloc[:, ~df_train.columns.isin(['compliance', 'ticket_id'])]
    y_train = df_train['compliance']
    X_valid = df_valid.iloc[:, ~df_valid.columns.isin(['compliance', 'ticket_id'])]

    scaler = preprocessing.StandardScaler().fit(X_train.values)
    X_train = scaler.transform(X_train.values) 
    X_valid = scaler.transform(X_valid.values) 
    
    clf = SGDClassifier(loss="log", penalty="l2")
    clf = GradientBoostingClassifier()
    scores = cross_val_score(clf, X_train, y_train, cv=5)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_valid)
    
    df = pd.Series(index=df_valid['ticket_id'], data=y_prob[:,1], name='compliance', dtype=np.float32)
    return df

In [5]:
blight_model()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':


Accuracy: 0.93 (+/- 0.01)


ticket_id
284932    0.131146
285362    0.018175
285361    0.066210
285338    0.050181
285346    0.080176
285345    0.050181
285347    0.068986
285342    0.465028
285530    0.017880
284989    0.023265
285344    0.062358
285343    0.017562
285340    0.017562
285341    0.062358
285349    0.080176
285348    0.050181
284991    0.023265
285532    0.023265
285406    0.023265
285001    0.035267
285006    0.017880
285405    0.017562
285337    0.023265
285496    0.061010
285497    0.049082
285378    0.018175
285589    0.023646
285585    0.049082
285501    0.066210
285581    0.018175
            ...   
376367    0.023265
376366    0.038120
376362    0.253847
376363    0.343729
376365    0.023265
376364    0.038120
376228    0.036861
376265    0.038120
376286    0.168698
376320    0.038120
376314    0.038120
376327    0.296663
376385    0.290533
376435    0.219110
376370    0.721105
376434    0.067505
376459    0.141601
376478    0.015584
376473    0.039762
376484    0.023145
376482    0.024680
37